In [1]:
import spacy

from cycontext import ConTextComponent, ConTextItem

# Clinical Text and Target Concepts
We'll start by defining a short, simple clinical texts for our use case. This text contains a target clinical concept: **"pneumonia"** and contextual information which needs to be extracted: **"There is no evidence"**.

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
text = "There is no evidence of pneumonia."

In [4]:
doc = nlp(text)

Target concepts should be stored as document spans in the attribute `Doc.ents`. For now, we'll manually define this span.

In [5]:
from spacy.tokens import Span

In [6]:
doc.ents = (Span(doc, 5, 6, label="EVIDENCE_OF_PNEUMONIA"),)

In [7]:
doc.ents

(pneumonia,)

In [13]:
context = ConTextComponent(nlp, rules="default")

In [14]:
context.item_data

[ConTextItem(literal='absence of', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='adequate to rule her out', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='adequate to rule him out', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='adequate to rule out', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='adequate to rule the patient out', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='although', category='NEG', pattern=None, rule='TERMINATE'),
 ConTextItem(literal='any other', category='NEG', pattern=None, rule='FORWARD'),
 ConTextItem(literal='apart from', category='NEG', pattern=None, rule='TERMINATE'),
 ConTextItem(literal='are negative', category='NEG', pattern=None, rule='BACKWARD'),
 ConTextItem(literal='are ruled out', category='NEG', pattern=None, rule='BACKWARD'),
 ConTextItem(literal='as a cause for', category='NEG', pattern=None, rule='TERMINATE'),
 ConTextItem(literal='as

# Applying ConText
Once we've added the ConTextItems, we call the ConTextComponent object directly on a Doc.

In [9]:
context(doc)

There is no evidence of pneumonia.

This adds the following attributes:
- `Doc._.context_graph`: An object containing the targets, modifiers, and relationships between them
- `Span._.modifiers`: A tuple added to each span which will contain the modifiers which modify each target entity
- Additional ConText attributes (optional)

## ConTextGraph
This object contains the main findings of the ConText algorithm. It handles applying the modifiers to the sentences, defining their scopes, and identifying target concepts which they modify.

In [11]:
doc._.context_graph

<ConTextGraph> with 1 targets and 0 modifiers

### Modifiers
The `modifiers` attribute is a list of `TagObject` objects, which are the result of a ConTextItem matching a span of text in `doc`. In this example, "no evidence of" and has a category of "DEFINITE_NEGATED_EXISTENCE", as defined by `context_item`.

In [12]:
doc._.context_graph.modifiers

[]

The `scope` object contains the span of text which is modified by the TagObject:

In [14]:
modifier = doc._.context_graph.modifiers[0]
modifier.scope

pneumonia.

### Targets
The `targets` attribute contains the list of entities in `doc.ents`:

In [15]:
doc._.context_graph.targets

(pneumonia,)

### Edges
This is the primary role of the ConText algorithm. Once modifiers and targets have been identified, any targets within the scope of a modifier are said to be **modified by** that modifier. In this example, this gives us the contextual semantic information that this entity is negated.

In [16]:
for target, modifier in doc._.context_graph.edges:
    print("[{0}] is modified by [{1}]".format(target, modifier))

[pneumonia] is modified by [<TagObject> [no evidence of, NEGATED_EXISTENCE]]


## Span._.modifiers
These relationships are also stored as a list in the `target._.modifiers` attribute. This allows us to identify all modifiers for a target entity:

In [17]:
for ent in doc.ents:
    print("{0} is modified by [{1}]".format(ent, ent._.modifiers))

pneumonia is modified by [(<TagObject> [no evidence of, NEGATED_EXISTENCE],)]


# Additional Span attributes
In addition to storing the results in the ConTextGraph, cycontext also sets several additional span-level attributes which contain the contextual information for that target. In the future this should be customizable, but this and similar attributes can also be set manually, as will be shown in a later notebook.

- is_negated: True if a target is modified by 'NEGATED_EXISTENCE', default False
- is_uncertain: True if a target is modified by 'POSSIBLE_EXISTENCE', default False
- is_historical: True if a target is modified by 'HISTORICAL', default False
- is_hypothetical: True if a target is modified by 'HYPOTHETICAL', default False
- is_family: True if a target is modified by 'FAMILY', default False

In [18]:
from spacy.tokens.span import Span

In [20]:
for ent in doc.ents:
    print(ent)
    print("is_negated: ", ent._.is_negated)
    print("is_uncertain: ", ent._.is_uncertain)
    print("is_historical: ", ent._.is_historical)
    print("is_hypothetical: ", ent._.is_hypothetical)
    print("is_family: ", ent._.is_family)
    

pneumonia
is_negated:  True
is_uncertain:  False
is_historical:  False
is_hypothetical:  False
is_family:  False


These attributes can be left out by setting `set_attrs` to `False` when initializing the `ConTextComponent`.

# Visualization
When building or explaining a clinical NLP system, it can be especially helpful to view visual representations of the entities and modifiers. We can use [spaCy's visualizers](https://spacy.io/usage/visualizers) to display this information.

The `visualize_ent` function displays targets and modifiers in a document in an NER-style form, highlighting the clinical entities and modifiers in a Doc:

In [21]:
from cycontext import viz
viz.visualize_ent(doc)

The `visualize_dep` function uses a dependency-parse style graphic to show the relationships between targets and modifiers:

In [22]:
viz.visualize_dep(doc)

# Define ConTextItem
In this sentence, **"pneumonia"** is negated. This negation is indicated by the contextual information. We can extract this by identifying the semantic modifier and relating it to the clinical entity.

In cycontext, we define modifiers in the `ConTextItem` class. We'll explain the ConTextItem class in more detail in another notebook. For now, we'll define this simple item:

In [9]:
context_item = ConTextItem("no evidence of", "NEGATED_EXISTENCE", rule="FORWARD")

We then add this ItemData to the context object in a list.

In [10]:
context.add([context_item])

# Next Steps
In the next notebook, we'll see how to add ConText to a spaCy pipeline to process multiple documents with different targets and modifiers.